In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from numpy import asarray
import glob
import cv2
import tensorflow as tf

#### Import Data

In [2]:
all_labels=pd.read_csv('train.csv')

In [3]:
image_list = []
for filename in glob.glob('SmallSample/*.jpg'):
    im=Image.open(filename)
    image_list.append(im)

In [4]:
image_names = []
for filename in glob.glob('SmallSample/*.jpg'):
    name=filename.strip('SmallSample\\')
    image_names.append(name)

In [5]:
image_names[:3]

['000a8f2d5c316a.jpg', '000be9acf46619.jpg', '000bef247c7a42.jpg']

####  use image names from small sample to determine species label

In [6]:
Labels_DF=pd.DataFrame(all_labels)
Labels_DF.rename(columns={'image':'file_name'}, inplace=True)

In [7]:
Labels_DF.head(2)

,file_name,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250


In [8]:
Image_DF=pd.DataFrame({
    'image': image_list,
    'file_name': image_names
    })

In [9]:
Image_DF.head(3)

,image,file_name
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000a8f2d5c316a.jpg
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000be9acf46619.jpg
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000bef247c7a42.jpg


In [10]:
test=Image_DF['file_name'][2]
Labels_DF.loc[Labels_DF['file_name']==test]

,file_name,species,individual_id
7,000bef247c7a42.jpg,humpback_whale,444d8894ccc8


In [11]:
## Join Dataframes

Big_DF=Image_DF.merge(Labels_DF, how='outer')

In [12]:
Big_DF.head(8)

,image,file_name,species,individual_id
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000a8f2d5c316a.jpg,bottlenose_dolphin,b9907151f66e
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000be9acf46619.jpg,beluga,afb9b3978217
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000bef247c7a42.jpg,humpback_whale,444d8894ccc8
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000c3d63069748.jpg,beluga,df94b15285b9
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000c476c11bad5.jpg,bottlenose_dolphin,b11b2404c7e3
5,<PIL.JpegImagePlugin.JpegImageFile image mode=...,00a198105cdc24.jpg,humpback_whale,da611bc6c389
6,<PIL.JpegImagePlugin.JpegImageFile image mode=...,00a3449831cb17.jpg,dusky_dolphin,e04885cf2a1e
7,<PIL.JpegImagePlugin.JpegImageFile image mode=...,00a4745e7218c4.jpg,bottlenose_dolpin,3b5cd491b73f


In [13]:
from sklearn.preprocessing import OrdinalEncoder
OE=OrdinalEncoder()
SE=OE.fit_transform(Big_DF[['species']])

In [14]:
Big_DF['Species_Category']=SE.astype(int)

In [15]:
Big_DF.head(5)

,image,file_name,species,individual_id,Species_Category
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000a8f2d5c316a.jpg,bottlenose_dolphin,b9907151f66e,2
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000be9acf46619.jpg,beluga,afb9b3978217,0
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000bef247c7a42.jpg,humpback_whale,444d8894ccc8,14
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000c3d63069748.jpg,beluga,df94b15285b9,0
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000c476c11bad5.jpg,bottlenose_dolphin,b11b2404c7e3,2


## Create function to augment images

In [16]:
def Augment(photo):
    photo.thumbnail((150,150))
    photo=photo.crop((0,0,150,150))
    #photo1=photo.convert(mode='L')
    #photo1=photo1.cv2.cvtColor(photo1, cv2.COLOR_GRAY2RGB)
    photo2=photo.transpose(Image.FLIP_TOP_BOTTOM)
    photo3=photo.rotate(-15)
    photo4=photo.rotate(-30)
    photo5=photo.rotate(-50)
    photo6=photo.rotate(15)
    photo7=photo.rotate(35)
    photo8=photo.rotate(55)
    photo9=photo.crop((0,0,125,125))
    photo9=photo9.crop((0,0,150,150))
    photo10=photo.crop((25,25,150,150))
    photo10=photo10.crop((0,0,150,150))
    List=[photo, photo2, photo3, photo4, photo5, photo6, photo7, photo8, photo9, photo10]
    
    return List
    
    

In [17]:
pic=Big_DF['image'][3]

List=Augment(pic)

## Create Function to normalize pixels

In [18]:
def Norm_Pix(list):
    pix_list=[]
    for each in list:
        pixels=asarray(each)
        pixels=pixels.astype(float)
        pixels/=255.0
        pix_list.append(pixels)
    
    return pix_list

In [19]:
New_List=Norm_Pix(List)

## Create function combining augmentation and normalization

In [20]:
def Transform(list):
    transformed_images=[]
    for each in list:
        Aug_list=Augment(each)
        norm_list=Norm_Pix(Aug_list)
        transformed_images.append(norm_list)
    return transformed_images
        

In [21]:
images= Big_DF['image'][:152]

Transformed_Images=Transform(images)

In [22]:
len(Transformed_Images)

152

###  returned 152 lists of 10 images each.  Need to add species and individual id to each image

In [23]:
Transformed_Images[75]

[array([[[0.35686275, 0.42745098, 0.57647059],
         [0.30588235, 0.38431373, 0.56862745],
         [0.3372549 , 0.42352941, 0.59607843],
         ...,
         [0.32156863, 0.41960784, 0.6       ],
         [0.30588235, 0.39607843, 0.58039216],
         [0.29411765, 0.38039216, 0.57254902]],
 
        [[0.54901961, 0.59607843, 0.68627451],
         [0.49803922, 0.54509804, 0.65882353],
         [0.43137255, 0.49411765, 0.61568627],
         ...,
         [0.34901961, 0.43921569, 0.61960784],
         [0.30980392, 0.40392157, 0.58431373],
         [0.30588235, 0.39215686, 0.58823529]],
 
        [[0.62352941, 0.64705882, 0.69803922],
         [0.56470588, 0.58823529, 0.6627451 ],
         [0.40784314, 0.4627451 , 0.6       ],
         ...,
         [0.2745098 , 0.36862745, 0.55294118],
         [0.25882353, 0.35294118, 0.54117647],
         [0.28627451, 0.37254902, 0.56470588]],
 
        ...,
 
        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.     

### Create function to add labels to each image

In [24]:
def Create_XYs(List_of_Transformed_Images):
    n=0
    X=[]
    y=[]
    for each_list in List_of_Transformed_Images:
        for each_image in each_list:
            i=each_image
            s=Big_DF['Species_Category'][n]
            X.append(i)
            y.append(s)
        n=n+1
    return X, y

In [25]:
variables=Create_XYs(Transformed_Images)

In [26]:
X,y=variables[0], variables[1]

##  Try Dataframe because above function seems to be making lists of arrays(X) and ints(y)

In [27]:
XY_DF=pd.DataFrame({
    'X_Values': X,
    'Y_Values':y
})

In [28]:
XY_DF.head(5)

,X_Values,Y_Values
0,"[[[0.6705882352941176, 0.6745098039215687, 0.6...",2
1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",2
2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",2
3,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",2
4,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",2


## convert X_values to tensorflow array

In [70]:
tf_files=[]
for each in XY_DF['X_Values']:
    ar=tf.keras.preprocessing.image.img_to_array(each)
    tf_files.append(ar)


In [71]:
test_files=[]
for each in XY_DF['X_Values']:
    ar=tf.keras.preprocessing.image.img_to_array(each)
    test_files.append(ar)

In [73]:
test_files

[array([[[0.67058825, 0.6745098 , 0.68235296],
         [0.68235296, 0.6862745 , 0.6901961 ],
         [0.6745098 , 0.6745098 , 0.68235296],
         ...,
         [0.654902  , 0.6666667 , 0.6627451 ],
         [0.6431373 , 0.654902  , 0.654902  ],
         [0.6392157 , 0.64705884, 0.6431373 ]],
 
        [[0.64705884, 0.6509804 , 0.6627451 ],
         [0.64705884, 0.6509804 , 0.6627451 ],
         [0.654902  , 0.65882355, 0.6666667 ],
         ...,
         [0.58431375, 0.59607846, 0.59607846],
         [0.5882353 , 0.6       , 0.59607846],
         [0.627451  , 0.63529414, 0.63529414]],
 
        [[0.60784316, 0.60784316, 0.6156863 ],
         [0.6       , 0.6       , 0.60784316],
         [0.60784316, 0.6117647 , 0.61960787],
         ...,
         [0.5921569 , 0.6039216 , 0.6117647 ],
         [0.627451  , 0.6392157 , 0.6392157 ],
         [0.6156863 , 0.627451  , 0.627451  ]],
 
        ...,
 
        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.     

In [30]:
XY_DF['tf_images']=tf_files

In [31]:
XY_DF

,X_Values,Y_Values,tf_images
0,"[[[0.6705882352941176, 0.6745098039215687, 0.6...",2,"[[[0.67058825, 0.6745098, 0.68235296], [0.6823..."
1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
4,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
...,...,...,...
1515,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",14,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1516,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",14,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1517,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",14,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1518,"[[[0.6549019607843137, 0.6705882352941176, 0.7...",14,"[[[0.654902, 0.67058825, 0.7176471], [0.658823..."


##  Build Classifier Model

In [32]:
import tensorflow as tf
from tensorflow import keras

In [84]:
X_train=tf_files[:120]

#y_train=XY_DF['Y_Values'][:120]
y_train=y[:120]


In [85]:
print(X_train[4].shape)
print(y_train[4].shape)
y_train

(150, 150, 3)
()


[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14]

In [76]:
model=keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[150,150,3]))
model.add(keras.layers.Dense(150, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(30, activation='softmax'))

In [82]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [86]:
model.fit(X_train, y_train, epochs=10)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'list'> containing values of types {"<class 'numpy.int32'>"})